# Constrained Shape Optimization

In [ ]:
# !pip install ngsolve --upgrade
# !pip install webgui_jupyter_widgets --upgrade
# !pip install matplotlib --upgrade

from shapeOptInductor import create_plots, update_plots, rot
import ngsolve as ngs
from ngsolve.webgui import Draw
import numpy as np
import matplotlib.pyplot as plt 


## 1 - Meshing

In [ ]:
from netgen.geom2d import SplineGeometry
from debug import *

thickness = 1e-2


def gen_mesh12(air_gap=4e-3, maxh=2e-3, debug=False):
    """Gives a triangular mesh"""
    # Global domain
    r = 0.04

    # Geometry definition
    e = 1.5e-2
    a = 1e-2
    ha = 1e-2
    ba = 1e-2
    geo = SplineGeometry()
    pnts = [
        (a / 2, air_gap / 2),  # p0
        (a / 2, e / 2),  # p1
        (a / 2 + ba, e / 2),  # p2
        (a / 2 + ba, air_gap / 2),  # p3
        (a + ba, air_gap / 2),  # p4
        (a + ba, e / 2 + a / 2),  # p5
        (-a - ba, e / 2 + a / 2), # p6
        (-a - ba, air_gap / 2), # p7
        (-a / 2 - ba, air_gap / 2), # p8
        (-a / 2 - ba, e / 2), # p9
        (-a / 2, e / 2), # p10
        (-a / 2, air_gap / 2), # p11
        (-a / 2, -air_gap / 2), # p12
        (-a / 2, -e / 2), # p13
        (-a / 2 - ba, -e / 2), # p14
        (-a / 2 - ba, -air_gap / 2), # p15
        (-a - ba, -air_gap / 2), # p16
        (-a - ba, -e / 2 - a / 2), # p17
        (a + ba, -e / 2 - a / 2),  # p18
        (a + ba, -air_gap / 2),  # p19
        (a / 2 + ba, -air_gap / 2),  # p20
        (a / 2 + ba, -e / 2),  # p21
        (a / 2, -e / 2),  # p22
        (a / 2, -air_gap / 2),  # p23

        (r, -r), # p100
        (r, r), # p101
        (-r, r), # p102
        (-r, -r), # p103
    ]

    (
        p0,
        p1,
        p2,
        p3,
        p4,
        p5,
        p6,
        p7,
        p8,
        p9,
        p10,
        p11,
        p12,
        p13,
        p14,
        p15,
        p16,
        p17,
        p18,
        p19,
        p20,
        p21,
        p22,
        p23,
        p100,
        p101,
        p102,
        p103,
    ) = [geo.AppendPoint(*pnt) for pnt in pnts]

     # List of lines with boundary conditions and domains
    # front, optimVert, default, domainHor, segment1, segment2, domainVert, arc, optimHor
    lines = [
        [["line", p11, p0], {"bc": "a", "leftdomain": 2, "rightdomain": 7}],
        [["line", p0, p1], {"bc": "b", "leftdomain": 2, "rightdomain": 4}],
        [["line", p1, p2], {"bc": "c", "leftdomain": 2, "rightdomain": 4}],
        [["line", p2, p3], {"bc": "d", "leftdomain": 2, "rightdomain": 4}],
        [["line", p3, p4], {"bc": "e", "leftdomain": 2, "rightdomain": 6}],
        [["line", p4, p5], {"bc": "f", "leftdomain": 2, "rightdomain": 6}],
        [["line", p5, p6], {"bc": "g", "leftdomain": 2, "rightdomain": 6}],
        [["line", p6, p7], {"bc": "h", "leftdomain": 2, "rightdomain": 6}],
        [["line", p7, p8], {"bc": "i", "leftdomain": 2, "rightdomain": 6}],
        [["line", p8, p9], {"bc": "j", "leftdomain": 2, "rightdomain": 5}],
        [["line", p9, p10], {"bc": "k", "leftdomain": 2, "rightdomain": 5}],
        [["line", p10, p11], {"bc": "l", "leftdomain": 2, "rightdomain": 5}],
        [["line", p23, p12], {"bc": "m", "leftdomain": 3, "rightdomain": 7}],
        [["line", p12, p13], {"bc": "n", "leftdomain": 3, "rightdomain": 5}],
        [["line", p13, p14], {"bc": "o", "leftdomain": 3, "rightdomain": 5}],
        [["line", p14, p15], {"bc": "p", "leftdomain": 3, "rightdomain": 5}],
        [["line", p15, p16], {"bc": "q", "leftdomain": 3, "rightdomain": 6}],
        [["line", p16, p17], {"bc": "r", "leftdomain": 3, "rightdomain": 6}],
        [["line", p17, p18], {"bc": "s", "leftdomain": 3, "rightdomain": 6}],
        [["line", p18, p19], {"bc": "t", "leftdomain": 3, "rightdomain": 6}],
        [["line", p19, p20], {"bc": "u", "leftdomain": 3, "rightdomain": 6}],
        [["line", p20, p21], {"bc": "v", "leftdomain": 3, "rightdomain": 4}],
        [["line", p21, p22], {"bc": "w", "leftdomain": 3, "rightdomain": 4}],
        [["line", p22, p23], {"bc": "x", "leftdomain": 3, "rightdomain": 4}],

        [["line", p3, p20], {"bc": "aa", "leftdomain": 6, "rightdomain": 4}],
        [["line", p23, p0], {"bc": "bb", "leftdomain": 7, "rightdomain": 4}],
        [["line", p12, p11], {"bc": "cc", "leftdomain": 5, "rightdomain": 7}],
        [["line", p15, p8], {"bc": "dd", "leftdomain": 6, "rightdomain": 5}],

        [["line", p100, p101], {"bc": "ee", "leftdomain": 6, "rightdomain": 0}],
        [["line", p101, p102], {"bc": "ff", "leftdomain": 6, "rightdomain": 0}],
        [["line", p102, p103], {"bc": "gg", "leftdomain": 6, "rightdomain": 0}],
        [["line", p103, p100], {"bc": "hh", "leftdomain": 6, "rightdomain": 0}], 
    ]

    # Append all lines to the geometry
    for line, props in lines:
        geo.Append(line, **props)

    # Optional matpotlib debug render
    if debug : 
        fig, axes = plt.subplots(2, 2, figsize=(15, 15))
        debug_geo(geo, axes[0, 0])
        debug_points(geo, 
                     # point_names is optional
                     point_names="p0,p1,p2,p3,p4,p5,p6,p7,p8,p9,p10,p11,p12,p13,p14,p15,p16,p17,p18,p19,p20,p21,p22,p23,p100,p101,p102,p103".split(","),
                     ax=axes[1, 0]
                     )
        debug_bc_names(geo, ax=axes[0, 1])
        debug_region_labels(geo, ax=axes[1, 1])
        plt.show()


    # Set materials and meshing parameters
    geo.SetMaterial(2, "core")
    geo.SetMaterial(3, "core")
    geo.SetMaterial(4, "coilR")
    geo.SetMaterial(5, "coilL")
    geo.SetMaterial(6, "air")
    geo.SetMaterial(7, "air")
    ngmesh = geo.GenerateMesh(maxh=maxh)
    return ngs.Mesh(ngmesh)


mesh = gen_mesh12(debug=True)
XiAir = mesh.MaterialCF({"air": 1})
XiCore = mesh.MaterialCF({"core": 1})
XiCoilR = mesh.MaterialCF({"coilR": 1})
XiCoilL = mesh.MaterialCF({"coilL": 1})
XiCoil = mesh.MaterialCF({"coilR": 1, "coilL": 1})

Draw(1 * XiAir + 2 * XiCoil + 3 * XiCore, mesh, radius=0.02)

## 2 - Computation of magnetic state

In [ ]:
# Frequency
f = 5e4  # Hz
omega = 2 * np.pi * f

# Magnetic
mu0 = 4e-7 * np.pi
mur = 1000
mu_iron = mur * mu0
delta = 0.1
mu_coil = np.exp( -1j * delta) * mu0  #  AC losses in the copper from the imaginary part of the permeability

# Current
nb_turn = 200  # Number of turn in the coil
Is = 2  # Source current intensity
js = nb_turn * Is / (ngs.Integrate(XiCoilL, mesh)) * (XiCoilR - XiCoilL)  # Source current density
coeff_losses = omega / 2 * thickness * (1 / mu_coil).imag

def magWeakFormComplex(a, a_):
    bf = ngs.grad(a_) * 1 / mu_iron * ngs.grad(a) * ngs.dx("core")
    bf += ngs.grad(a_) * 1 / mu_coil * ngs.grad(a) * ngs.dx("coilL")
    bf += ngs.grad(a_) * 1 / mu_coil * ngs.grad(a) * ngs.dx("coilR")
    bf += ngs.grad(a_) * 1 / mu0 * ngs.grad(a) * ngs.dx("air")
    lf = a_ * js * ngs.dx("coilL")
    lf += a_ * js * ngs.dx("coilR")
    return bf, lf


def solveStateComplex(fes):
    a, a_ = fes.TnT()
    bf, f = magWeakFormComplex(a, a_)
    K, F = ngs.BilinearForm(fes), ngs.LinearForm(fes)
    K += bf
    F += f
    K.Assemble()
    F.Assemble()
    gf = ngs.GridFunction(fes)
    Kinv = K.mat.Inverse(freedofs=fes.FreeDofs(), inverse="pardiso")  # Hermitian ?
    gf.vec.data = Kinv * F.vec
    return gf, Kinv


fes = ngs.H1(mesh, order=1, dirichlet="ee|ff|gg|hh", complex=True)
state, Kinv = solveStateComplex(fes)
Draw(state.real, mesh, radius=0.02)

In [ ]:
def Inductance(a, mesh):
    rel = XiAir / mu0 + XiCoil / mu_coil + XiCore / mu_iron
    return 2 * thickness / (2 * Is**2) * ngs.Integrate(rel.real * ngs.Norm(ngs.grad(a)) ** 2, mesh)


def Losses(a, mesh):
    return ngs.Integrate(coeff_losses * XiCoil * ngs.Norm(ngs.grad(a)) ** 2, mesh)


# For the augmented lagrangian
L_target = 1e-3


def Constraint(a, mesh):
    return Inductance(a, mesh) - L_target


def CostFunction(a, l, b, constraint, mesh):
    return (Losses(a, mesh) + l * constraint + 0.5 * b * constraint**2).real


In [ ]:
fes = ngs.H1(mesh, order=1, dirichlet="ee|ff|gg|hh", complex=True)
state, Kinv = solveStateComplex(fes)

print(f"The inductance is {1e3 * np.absolute(Inductance(state, mesh)):.3f} mH")
print(f"The losses amounts to {Losses(state, mesh):.3f} W")
constraint = Constraint(state, mesh)
print(f"The cost function amounts to {CostFunction(state, 1, 1, constraint, mesh):.3f} W")


## 3 - Computation of adjoint states and shape derivatives

In [ ]:
def solveAdjointLosses(a0, Kinv):
    """Solves the adjoint equation for the losses inside the coil"""
    fes = a0.space
    p, p_ = fes.TnT()
    F = ngs.LinearForm(fes)
    F += - 2 * coeff_losses * ngs.InnerProduct(ngs.grad(a0), ngs.grad(p_)) * ngs.dx("coilR|coilL")
    F.Assemble()
    gf = ngs.GridFunction(fes)
    gf.vec.data = Kinv.H * F.vec
    return gf


def solveAdjointInductance(a0, Kinv):
    """Solves the adjoint equation for the inductance"""
    fes = a0.space
    p, p_ = fes.TnT()
    F = ngs.LinearForm(fes)
    rel = (1 / mu0) * XiAir + (1 / mu_coil) * XiCoil + (1 / mu_iron) * XiCore
    coeff_induc =  2 * thickness / (2 * Is**2) * rel
    F += -2 * coeff_induc * ngs.InnerProduct(ngs.grad(a0), ngs.grad(p_)).real * ngs.dx
    F.Assemble()
    gf = ngs.GridFunction(fes)
    gf.vec.data =  Kinv * F.vec
    return gf


def computeLossesShapeDerivative(mesh, VEC_complex, VEC_real, a0, p0):
    """Shape derivative for the energy-based losses inside the coil"""
    X = VEC_complex.TestFunction()

    rel = (1 / mu0) * XiAir + (1 / mu_coil) * XiCoil + (1 / mu_iron) * XiCore
    Id = ngs.CoefficientFunction((1, 0, 0, 1), dims=(2, 2))  # Identity matrix
    dA = ngs.div(X) * Id - ngs.grad(X) - ngs.grad(X).trans

    dLOmega = ngs.LinearForm(VEC_complex)
    dLOmega += coeff_losses * XiCoil * ngs.InnerProduct(dA * ngs.grad(a0), ngs.grad(a0)) * ngs.dx("coilL|coilR")
    dLOmega += -js * XiCoil * ngs.div(X) * p0 * ngs.dx("coilL|coilR")
    dLOmega += ngs.InnerProduct(rel * dA * ngs.grad(a0), ngs.grad(p0)) * ngs.dx("coilL|coilR")
    dLOmega += ngs.InnerProduct(rel * dA * ngs.grad(a0), ngs.grad(p0)) * ngs.dx("air|core")

    dLOmega.Assemble()
    dJ = ngs.GridFunction(VEC_real)
    dJ.vec.FV().NumPy()[:] = np.real(dLOmega.vec.FV().NumPy()[:])
    return dJ


def computeInductanceShapeDerivative(mesh, VEC_complex, VEC_real, a0, p0):
    """Shape derivative for the inductance"""
    X = VEC_complex.TestFunction()

    rel = (1 / mu0) * XiAir + (1 / mu_coil) * XiCoil + (1 / mu_iron) * XiCore
    Id = ngs.CoefficientFunction((1, 0, 0, 1), dims=(2, 2))  # Identity matrix
    coeff_induc = 2 * thickness / (2 * Is**2) * rel
    dA = ngs.div(X) * Id - ngs.grad(X) - ngs.grad(X).trans

    dLOmega = ngs.LinearForm(VEC_complex)
    dLOmega += coeff_induc * ngs.InnerProduct(dA * ngs.grad(a0), ngs.grad(a0)) * ngs.dx
    dLOmega += -js * ngs.div(X) * p0 * ngs.dx
    dLOmega += ngs.InnerProduct(rel * dA * ngs.grad(a0), ngs.grad(p0)) * ngs.dx
    dLOmega.Assemble()

    dJ = ngs.GridFunction(VEC_real)
    dJ.vec.FV().NumPy()[:] = np.real(dLOmega.vec.FV().NumPy()[:])
    return dJ


def computeShapeDerivative(mesh, VEC_complex, VEC_real, state, adjoint_losses, adjoint_inductance, l, b):
    """Shape derivative for the cost function"""
    dJOmegaInductance = computeInductanceShapeDerivative(mesh, VEC_complex, VEC_real, state, adjoint_inductance)
    constraint = Constraint(state, mesh)
    dJOmega = computeLossesShapeDerivative(mesh, VEC_complex, VEC_real, state, adjoint_losses)
    dJOmega.vec.FV().NumPy()[:] += l * dJOmegaInductance.vec.FV().NumPy()[:]
    dJOmega.vec.FV().NumPy()[:] += b * constraint * dJOmegaInductance.vec.FV().NumPy()[:]
    return dJOmega


In [ ]:
maxh = 10e-4
mesh = gen_mesh12(air_gap=4e-3, maxh=maxh)

fes = ngs.H1(mesh, order=1, dirichlet="ee|ff|gg|hh", complex=True)
VEC_complex = ngs.VectorH1(mesh, complex=True)
VEC_real = ngs.VectorH1(mesh)

state, Kinv = solveStateComplex(fes)
adjoint_losses = solveAdjointLosses(state, Kinv)
adjoint_inductance = solveAdjointInductance(state, Kinv)

dJOmega = computeShapeDerivative(mesh, VEC_complex, VEC_real, state, adjoint_losses, adjoint_inductance, 1, 1)

Draw(adjoint_inductance, radius=0.02)
Draw(adjoint_losses.real, mesh, radius=0.02)  # Note that the imag part should be 0



## 4 - Computation of descent direction (deformation field)

In [ ]:
def SolveDeformationEquation1(mesh, fX):
    VEC = ngs.VectorH1(
        mesh,
        order=1,
        dirichlet="ee|ff|gg|hh", # Border of domain
        dirichletx="p|j|n|l|x|b|d|v|dd|cc|bb|aa|f|h|r|t", # See bc names in the figure in part 1
        dirichlety="o|k|c|w|g|s",
    )
    PHI, X = VEC.TnT()

    # H1 dot product
    B = ngs.BilinearForm(VEC)
    B += ngs.InnerProduct(ngs.grad(X), ngs.grad(PHI)) * ngs.dx + ngs.InnerProduct(X, PHI) * ngs.dx
    B.Assemble()

    gfX = ngs.GridFunction(VEC)
    gfX.vec.data = -B.mat.Inverse(VEC.FreeDofs()) * fX.vec
    return gfX


def SolveDeformationEquation2(mesh, fX):
    VEC = ngs.VectorH1(
        mesh,
        order=1,
        dirichlet="ee|ff|gg|hh",
        dirichletx="p|j|n|l|x|b|d|v|dd|cc|bb|aa|f|h|r|t",
        dirichlety="o|k|c|w|g|s",
        definedon="air|core",
    )
    PHI, X = VEC.TnT()

    # H1 dot product
    B = ngs.BilinearForm(VEC)
    B += ngs.InnerProduct(ngs.grad(X), ngs.grad(PHI)) * ngs.dx + ngs.InnerProduct(X, PHI) * ngs.dx
    B.Assemble()

    gfX = ngs.GridFunction(VEC)
    gfX.vec.data = -B.mat.Inverse(VEC.FreeDofs()) * fX.vec
    return gfX

def SolveDeformationEquation3(mesh, fX):
    VEC = ngs.VectorH1(
        mesh,
        order=1,
        dirichlet="ee|ff|gg|hh|g|s|p|j|n|l|x|b|d|v|dd|cc|bb|aa|o|k|c|w",
        dirichletx="m|u|f|h|r|t",
        definedon="air|core",
    )
    PHI, X = VEC.TnT()

    # H1 dot product
    B = ngs.BilinearForm(VEC)
    B += ngs.InnerProduct(ngs.grad(X), ngs.grad(PHI)) * ngs.dx + ngs.InnerProduct(X, PHI) * ngs.dx
    B.Assemble()

    gfX = ngs.GridFunction(VEC)
    gfX.vec.data = -B.mat.Inverse(VEC.FreeDofs()) * fX.vec
    return gfX


fes = ngs.H1(mesh, order=1, dirichlet="ee|ff|gg|hh", complex=True)
VEC_complex = ngs.VectorH1(mesh, complex=True)
VEC_real = ngs.VectorH1(mesh)

print("Deformation field without dirichlet conditions")
dJ = ngs.GridFunction(VEC_real)
dJ.vec.FV().NumPy()[:] = dJOmega.vec.FV().NumPy()[:]
Draw(dJ, mesh, vectors={"grid_size": 40}, radius=0.02)

print("SolveDeformationEquation1 :\n Deformation field with dirichlet conditions for the field to be tangent to the coil")
descent_direction = SolveDeformationEquation1(mesh, dJ)
Draw(descent_direction.real, mesh, vectors={"grid_size": 40}, radius=0.02)

print("SolveDeformationEquation2 :\n Deformation field with dirichlet conditions for the field to vanish inside the coil and tangent at the boundary")
descent_direction = SolveDeformationEquation2(mesh, dJ)
Draw(descent_direction.real, mesh, vectors={"grid_size": 40}, radius=0.02)

print("SolveDeformationEquation3 :\n Deformation field with dirichlet conditions for the field to vanish inside the coil and at the boundary")
descent_direction = SolveDeformationEquation3(mesh, dJ)
Draw(descent_direction.real, mesh, vectors={"grid_size": 40}, radius=0.02)



## 5 - Taylor tests

In [ ]:
from mmglib import copy_ngmesh

def TaylorTest(cost, compute_adjoint, compute_shape_derivative, SolveDeformationEquation, title="Taylor test", maxh=1.5e-3, to_draw=False, ax=None):
    mesh = gen_mesh12(air_gap=2e-3, maxh=maxh)
    fes = ngs.H1(mesh, order=1, dirichlet="ee|ff|gg|hh", complex=True)
    VEC_complex = ngs.VectorH1(mesh, complex=True)
    VEC_real = ngs.VectorH1(mesh)
    a0, Kinv = solveStateComplex(fes)
    p0 = compute_adjoint(a0, Kinv)
    dJOmega = compute_shape_derivative(mesh, VEC_complex, VEC_real, a0, p0)

    nb_sample = 20
    exponent_min = 2.6
    exponent_max = 10
    reminders = []

    X = []

    # For transported mesh
    meshT = ngs.Mesh(copy_ngmesh(mesh.ngmesh))
    fesT = ngs.H1(meshT, order=1, dirichlet="ee|ff|gg|hh", complex=True)
    VEC_realT = ngs.VectorH1(meshT)

    direction = ngs.GridFunction(VEC_realT)
    direction = SolveDeformationEquation(mesh, dJOmega)
    direction.vec.FV().NumPy()[:] = direction.vec.FV().NumPy() / (1e-10 + np.max(np.abs(direction.vec.FV().NumPy())))

    if to_draw : scene = Draw(ngs.Norm(direction), meshT, radius=0.02)

    for i in range(nb_sample, 0, -1):
        # Abscissa axis in log scale
        t = 10 ** (-(i / nb_sample * (exponent_max - exponent_min) + exponent_min))
        X.append(t)

        # Finite difference
        displacement = ngs.GridFunction(VEC_realT)
        displacement.Set(t * direction)
        
        meshT.SetDeformation(displacement)
        aT, _ = solveStateComplex(fesT)
        difference = cost(aT, meshT) - cost(a0, mesh)
        if to_draw : scene.Redraw(ngs.Norm(direction), meshT, radius=0.02)

        # Reminder for analytic shape derivative
        reminder_value = np.abs(difference - ngs.InnerProduct(dJOmega.vec, t * direction.vec))
        reminders.append(reminder_value)

    Y = np.power(np.array(X), 2)
    # Create a new figure if no ax is provided
    if ax is None:
        fig, ax = plt.subplots(figsize=(8, 8))

    ax.plot(X, reminders, label="reminder", marker="s")
    ax.plot(X, Y, label="y = t^2", linestyle="--", color="gray")
    ax.set_xscale("log")
    ax.set_yscale("log")
    ax.set_xlabel("t")
    ax.set_ylabel("Reminder")
    ax.set_title(title)
    ax.legend()
    ax.grid(True, which="both", ls="--")

    if ax is None:
        plt.tight_layout()
        plt.show()

maxh = 3e-3

print("Taylor test without any SolveDeformationEquation function")
fig, axes = plt.subplots(1, 2, figsize=(14, 6))
TaylorTest(Losses, solveAdjointLosses, computeLossesShapeDerivative, lambda mesh, dJOmega : dJOmega, ax=axes[0], title="Taylor test for the losses", maxh=maxh, to_draw=True)
TaylorTest(Inductance, solveAdjointInductance, computeInductanceShapeDerivative, lambda mesh, dJOmega : dJOmega, ax=axes[1], title="Taylor test for the inductance", maxh=maxh)
plt.show()

print("Taylor test for SolveDeformationEquation1")
fig, axes = plt.subplots(1, 2, figsize=(14, 6))
TaylorTest(Losses, solveAdjointLosses, computeLossesShapeDerivative, SolveDeformationEquation1, ax=axes[0], title="Taylor test for the losses", maxh=maxh, to_draw=True)
TaylorTest(Inductance, solveAdjointInductance, computeInductanceShapeDerivative, SolveDeformationEquation1, ax=axes[1], title="Taylor test for the inductance", maxh=maxh)
plt.show()

print("Taylor test for SolveDeformationEquation2")
fig, axes = plt.subplots(1, 2, figsize=(14, 6))
TaylorTest(Losses, solveAdjointLosses, computeLossesShapeDerivative, SolveDeformationEquation2, ax=axes[0], title="Taylor test for the losses", maxh=maxh, to_draw=True)
TaylorTest(Inductance, solveAdjointInductance, computeInductanceShapeDerivative, SolveDeformationEquation2, ax=axes[1], title="Taylor test for the inductance", maxh=maxh)
plt.show()

print("Taylor test for SolveDeformationEquation3")
fig, axes = plt.subplots(1, 2, figsize=(14, 6))
TaylorTest(Losses, solveAdjointLosses, computeLossesShapeDerivative, SolveDeformationEquation3, ax=axes[0], title="Taylor test for the losses", maxh=maxh, to_draw=True)
TaylorTest(Inductance, solveAdjointInductance, computeInductanceShapeDerivative, SolveDeformationEquation3, ax=axes[1], title="Taylor test for the inductance", maxh=maxh)
plt.show()


## 6 - Testing mesh adaptation with mmg

In [ ]:
# Initializations
mesh = gen_mesh12(air_gap=4e-3, maxh=10e-4)
fes = ngs.H1(mesh, order=1, dirichlet="ee|ff|gg|hh", complex=True)
VEC_complex = ngs.VectorH1(mesh, complex=True)
VEC_real = ngs.VectorH1(mesh)

l = 1e-2
b = 1e-1

# State and cost function
state, Kinv = solveStateComplex(fes)
constraint = Constraint(state, mesh)
Jold = CostFunction(state, l, b, constraint, mesh)

# Adjoints, shape derivate then descent direction
adjoint_losses = solveAdjointLosses(state, Kinv)
adjoint_inductance = solveAdjointInductance(state, Kinv)
dJOmega = computeShapeDerivative(mesh, VEC_complex, VEC_real, state, adjoint_losses, adjoint_inductance, 1, 1)

descent_direction = SolveDeformationEquation1(mesh, dJOmega)
currentNorm = ngs.Norm(descent_direction.vec)
derivative = ngs.InnerProduct(dJOmega.vec, descent_direction.vec)

if derivative > 0:
    raise ValueError("derivative positive :(")


def move_ngmesh_2D(displ, mesh):
    mat_displ = displ.vec.FV().NumPy()
    nb_points = len(mat_displ) // 2
    for i, point in enumerate(mesh.ngmesh.Points()):
        vx = mat_displ[i]
        vy = mat_displ[i + nb_points]
        point[0] += vx
        point[1] += vy
    mesh.ngmesh.Update()


from mmglib import run_adapt, copy_ngmesh

# print("Initial mesh")
# state, Kinv = solveStateComplex(fes)
# Draw(ngs.Norm(rot(state)), mesh, radius=0.02)

# print("copy of Initial mesh")
# copied_ngmesh = copy_ngmesh(mesh.ngmesh)
# copied_mesh = ngs.Mesh(copied_ngmesh)
# Draw(ngs.Norm(rot(state)), copied_mesh, radius=0.02)

print("Moved mesh")
descent_direction.vec.data = 4e-6 * descent_direction.vec
move_ngmesh_2D(descent_direction, mesh)
# mesh.ngmesh.OptimizeMesh2d() # Does not work

# Test the descent of the cost
fes = ngs.H1(mesh, order=1, dirichlet="ee|ff|gg|hh", complex=True)
state, Kinv = solveStateComplex(fes)
constraint = Constraint(state, mesh)
Jnew = CostFunction(state, l, b, constraint, mesh)

if Jnew < Jold:
    print("J decreased")
else:
    print("J increased")
Draw(ngs.Norm(rot(state)), mesh, radius=0.02)

# print("copy of Initial mesh")
# Draw(ngs.Norm(rot(state)), copied_mesh, radius=0.02)

print("Optimized moved mesh")
new_ngmesh, return_code = run_adapt(mesh.ngmesh, hausd=3e-6, hmax=2e-3)
new_mesh = ngs.Mesh(new_ngmesh)
fes = ngs.H1(new_mesh, order=1, dirichlet="ee|ff|gg|hh", complex=True)
state, Kinv = solveStateComplex(fes)
Draw(ngs.Norm(rot(state)), new_mesh, radius=0.02)

print(f"The inductance is {1e3 * np.absolute(Inductance(state, new_mesh)):.3f} mH")
print(f"The losses amounts to {Losses(state, new_mesh):.3f} W")
constraint = Constraint(state, new_mesh)
print(f"The cost function amounts to {CostFunction(state, 1, 1, constraint, new_mesh):.3f} W")

# print("Recover initial mesh")
# descent_direction.vec.data = -descent_direction.vec
# move_ngmesh_2D(descent_direction, mesh)
# fes = ngs.H1(mesh, order=1, dirichlet="arc|segment2|domainVert", complex=True)
# state, tangent_matrix = solveStateComplex(fes)
# Draw(ngs.Norm(rot(state)), mesh, radius=0.02)


## 7 - Optimization loop


In [ ]:
def optimize(mesh, maxh, SolveDeformationEquation, iter_max=1000, iter_softmax=1000, 
             iter_tol=800, l=1e-2, b=1e-1, btarget=1e6, minstep=1e-11, maxstep=0.5, 
             step=0.1, iter_init=0, use_mmg=False, mmg_hausd=1e-5, mmg_hmax=3e-3, mmg_hmin=5e-4):
    fes = ngs.H1(mesh, order=1, dirichlet="ee|ff|gg|hh", complex=True)
    VEC_complex = ngs.VectorH1(mesh, order=1, complex=True)
    VEC_real = ngs.VectorH1(mesh, order=1)

    # Initialize data arrays and plots
    cost_values = []
    losses_values = []
    inductance_values = []
    num_plots = 3
    fig, axes, hdisplay = create_plots(num_plots)
    curve_labels = [
        ["Cost Function (W)"],
        ["Losses Conductor (W)"],
        ["Inductance (mH)"],
    ]

    l_values = []
    b_values = []
    num_plots = 2
    figBis, axesBis, hdisplayBis = create_plots(num_plots)
    curve_labels_bis = [
        ["l"],
        ["b"],
    ]


    # Algorithmic parameters
    converged = False
    iter_i = iter_init
    eps = 1e-10
    export_count = 0

    # Initializing state, adjoints, cost function
    state, Kinv = solveStateComplex(fes)
    adjoint_losses = solveAdjointLosses(state, Kinv)
    adjoint_inductance = solveAdjointInductance(state, Kinv)
    dJOmega = computeShapeDerivative(mesh, VEC_complex, VEC_real, state, adjoint_losses, adjoint_inductance, l, b)
    constraint = Constraint(state, mesh)
    Jnew = CostFunction(state, l, b, constraint, mesh)

    scene = Draw(ngs.Norm(rot(state)), mesh, radius=0.02)
    # scene2 = Draw(rot(state), mesh, radius=0.02, vectors={"grid_size": 40})
    # scene3 = Draw(ngs.Norm(rot(state)), mesh, radius=0.02)

    # Optimization loop
    while not converged and iter_i < iter_max:
        # Step 1: Update fe spaces
        fes = ngs.H1(mesh, order=1, dirichlet="ee|ff|gg|hh", complex=True)
        VEC_complex = ngs.VectorH1(mesh, order=1, complex=True)
        VEC_real = ngs.VectorH1(mesh, order=1)

        # Step 2: Update the state and adjoint state
        state, Kinv = solveStateComplex(fes)
        adjoint_losses = solveAdjointLosses(state, Kinv)
        adjoint_inductance = solveAdjointInductance(state, Kinv)

        constraintold = Constraint(state, mesh)
        Jold = CostFunction(state, l, b, constraintold, mesh)

        # Step 2.5: Update the data arrays
        cost_values.append(Jold)
        losses_values.append(Losses(state, mesh))
        inductance_values.append(1e3 * Inductance(state, mesh))
        data = [
            [cost_values],
            [losses_values],
            [inductance_values],
        ]
        update_plots(fig, axes, hdisplay, data, curve_labels)

        l_values.append(l)
        b_values.append(b)
        data = [
            [l_values],
            [b_values],
        ]
        update_plots(figBis, axesBis, hdisplayBis, data, curve_labels_bis)

        # Step 2.6: Export if inductance cross 1mH
        if iter_i > iter_init + 3 and np.sign(inductance_values[-1] - 1) != np.sign(inductance_values[-2] - 1):
            print(f"Inductance crossed 1mH {iter_i=} inductance={inductance_values[-1]} mH losses={losses_values[-1]} W")
            # mesh.ngmesh.Save(f"../outputs/freeform_optimized_mesh_{export_count}.vol")
            # fig.savefig(f"../outputs/freeform_cv_{export_count}.png")
            # figBis.savefig(f"../outputs/freeform_cv2_{export_count}.png")
            # gf = ngs.GridFunction(ngs.L2(mesh))
            # gf.Set(ngs.Norm(rot(state)))
            # Draw(gf, mesh, radius=0.02, filename=f"../outputs/freeform_result_{export_count}.html", show=False)
            export_count += 1
            if iter_i >= iter_softmax:
                # fig.savefig("../outputs/freeform_simplified_cv.png")
                # figBis.savefig("../outputs/freeform_simplified_cv.png")
                plt.close(fig)
                plt.close(figBis)
                return mesh, iter_i, l, b, step

        # Step 3: Compute shape derivative
        dJOmega = computeShapeDerivative(mesh, VEC_complex, VEC_real, state, adjoint_losses, adjoint_inductance, l, b)

        # Step 4: Find a descent direction
        descent_direction = SolveDeformationEquation(mesh, dJOmega)
        derivative = ngs.InnerProduct(dJOmega.vec, descent_direction.vec)

        if derivative > 1e-10:
            raise ValueError(f"derivative positive :( {derivative=}")

        # Step 4.5: Scene redraw
        scene.Redraw(ngs.Norm(rot(state)), mesh, radius=0.02)

        # Step 5: Find suitable step with a line search
        Jold = Jnew
        copied_ngmesh = copy_ngmesh(mesh.ngmesh)
        copied_mesh = ngs.Mesh(copied_ngmesh)
        descent_direction_old = ngs.GridFunction(VEC_real)
        descent_direction_old.vec.data = descent_direction.vec
        i = 0
        imax = 5
        while i < imax:
            descent_direction.vec.data = step * maxh / (currentNorm + eps) * descent_direction_old.vec
            move_ngmesh_2D(descent_direction, mesh)
            return_code = 1

            # Mesh adaptation with mmg
            if use_mmg and iter_i % 5 == 0:
                new_ngmesh, return_code = run_adapt(mesh.ngmesh, hausd=mmg_hausd, hmax=mmg_hmax, hmin=mmg_hmin)
                mesh = ngs.Mesh(new_ngmesh)
                fes = ngs.H1(mesh, order=1, dirichlet="ee|ff|gg|hh", complex=True)
                VEC_complex = ngs.VectorH1(mesh, order=1, complex=True)
                VEC_real = ngs.VectorH1(mesh, order=1)

            # Compute new cost
            state, Kinv = solveStateComplex(fes)
            constraint = Constraint(state, mesh)
            Jnew = CostFunction(state, l, b, constraint, mesh)

            if return_code == 1 and Jnew < Jold + (iter_i < iter_tol) * np.abs(Jold):
                step = min(maxstep, 1.2 * step)
                break
            else:
                if (return_code != 1):
                    print("mmg fail")
                print("Line search fail")
                step = max(minstep, 0.5 * step)
                mesh = copied_mesh
                Jnew = Jold
            i += 1

        # Update augmented lagrangian parameter
        if i == imax:
            pass
        else:
            l = l + b * Constraint(state, mesh)
            if b < btarget:
                b = min(1.1 * b, btarget)

        # Step 6: Stopping criteria
        if step <= minstep:
            print("Stoping criteria")
            converged = True
        iter_i += 1

    if iter_i == iter_max:
        print("Gradient descent max iteration reached")

    fig.savefig("../outputs/freeform_simplified_cv.png")
    figBis.savefig("../outputs/freeform_simplified_cv.png")
    plt.close(fig)
    plt.close(figBis)
    return mesh, iter_i, l, b, step

In [ ]:
print("Optimization without mmg")
maxh = 10e-4
mesh = gen_mesh12(air_gap=4e-3, maxh=maxh)
new_mesh, iter_i, l, b, step = optimize(mesh, maxh, SolveDeformationEquation1, iter_softmax=50, iter_max=100, mmg_hausd=1e-6, mmg_hmax=10e-4, mmg_hmin=1e-4, b=1e4)

In [ ]:
print("Optimization with mmg")
maxh = 10e-4
mesh = gen_mesh12(air_gap=4e-3, maxh=maxh)
new_mesh2, iter_i, l, b, step = optimize(mesh, maxh, SolveDeformationEquation1, use_mmg=True, iter_softmax=300, iter_max=500, mmg_hausd=1e-6, mmg_hmax=10e-4, mmg_hmin=1e-4, b=1e4)

## 8 - Post-processing / Save results

In [ ]:
fes = ngs.H1(new_mesh2, order=1, dirichlet="ee|ff|gg|hh", complex=True)
state, Kinv = solveStateComplex(fes)

print(f"The optimization converged in {iter_i} iterations")
print(f"The inductance is {1e3 * np.absolute(Inductance(state, new_mesh2))} mH")
print(f"The losses amounts to {Losses(state, new_mesh2)} W")


In [ ]:
# new_mesh2.ngmesh.Save("../outputs/freeform_simplified_optimized_final.vol")
# gf = ngs.GridFunction(ngs.L2(new_mesh2))
# gf.Set(ngs.Norm(rot(state)))
# Draw(gf, new_mesh2, radius=0.02, filename="../outputs/freeform_simplified_optimized_result.html")
# Draw(1 * XiAir + 2 * XiCoil + 3 * XiCore, new_mesh2, radius=0.02, filename="../outputs/freeform_simplified_optimized_result_colored.html")
